### Basic Imports

In [220]:
import nbimporter
from typing import Dict, Tuple, List
import numpy as np
from tqdm import tqdm

### Env Vars

In [221]:
QUBITS_NUM = 4 
N = 16
K = 4
NUM_SHOTS = 1024
NUM_ITERATIONS = 10

approximated_energies_dict = { "approximated_eneriges_0": [],
"approximated_eneriges_1":[],
"approximated_eneriges_2": [],
"approximated_eneriges_3": [] }

w_vector = np.asarray([4,3,2,1])

### Simulator Backend

In [222]:
from qiskit import Aer
from qiskit.utils import QuantumInstance, algorithm_globals

seed = 50
algorithm_globals.random_seed = seed

simulator_backend = Aer.get_backend('qasm_simulator')

### BFGS Optimizer

In [223]:
from scipy.optimize import minimize

### K input states (computational basis)

In [224]:
from utiles import *

In [225]:
input_states = get_first_k_eigenvectors_from_n_computational_basis(K, N)

### Ansatz State

In [226]:
from ansatz_circuit_item2 import get_full_variational_quantum_circuit

In [227]:
init_circuit_params = {
    "thetas": np.random.uniform(low=0, high=2*np.pi, size=8),
    "phis": np.random.uniform(low=0, high=2*np.pi, size=4),
    "D1": 2,
    "D2": 8
}

In [228]:
def prepare_circuit_params(thetas) -> Dict:
     return {
    "thetas": thetas[4:],
    "phis": thetas[:4],
    "D1": 2,
    "D2": 8
     }

In [229]:
def get_ansatz_state(circuit_params, input_state):
    circuit_params_with_input_state = {**circuit_params, "input_state": input_state}
    return get_full_variational_quantum_circuit(**circuit_params_with_input_state)

## Expectation Value

### convert hamiltonian to pauli strings

In [230]:
def transfrom_hamiltonian_into_pauli_strings(hamiltonian) -> List:
    pauli_operators = hamiltonian.to_pauli_op().settings['oplist']
    pauli_coeffs = list(map(lambda pauli_operator: pauli_operator.coeff, pauli_operators))
    pauli_strings = list(map(lambda pauli_operator: pauli_operator.primitive, pauli_operators))
    return pauli_coeffs, pauli_strings

### pauli string reduction to sigma_z's

In [231]:
from qiskit.circuit.library.standard_gates import HGate, SGate
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister

In [232]:
reducing_to_pauli_z_mapping = {
    'I': 'I',
    'Z': 'Z',
    'X': 'Z',
    'Y': 'Z'
} 

In [233]:
def reduce_pauli_matrixes_into_sigma_z(pauli_string) -> str:
    reduced_pauli_string = ""
    for matrix_index in range(QUBITS_NUM):
        pauli_matrix = str(pauli_string[matrix_index])
        reduced_pauli_matrix = reducing_to_pauli_z_mapping[pauli_matrix]
        reduced_pauli_string = reduced_pauli_matrix + reduced_pauli_string
    
    return reduced_pauli_string

In [234]:
def add_layer_of_gates_for_reducing_paulis_to_sigma_z(pauli_string, quantum_circuit):
    quantum_registers = QuantumRegister(QUBITS_NUM, name="qubit")
    additional_circuit_layer = QuantumCircuit(quantum_registers)
    
    for quantum_register_index, pauli_matrix in enumerate(pauli_string):
        if pauli_matrix == "X":
            additional_circuit_layer.append(HGate(), [quantum_registers[quantum_register_index]])
        if pauli_string == "Y":
            additional_circuit_layer.append(HGate(), [quantum_registers[quantum_register_index]])
            additional_circuit_layer.append(SGate(), [quantum_registers[quantum_register_index]])
                
    extended_quantum_circuit = quantum_circuit.compose(additional_circuit_layer)
    return extended_quantum_circuit

### probabilities distribution

In [235]:
def get_probability_distribution(counts: Dict) -> Dict:
    proba_distribution = {state: (count / NUM_SHOTS) for state, count in counts.items()}
    return proba_distribution

def calculate_probabilities_of_measurments_in_computational_basis(quantum_state_circuit) -> Dict:
    quantum_state_circuit.measure_all()
    
    transpiled_quantum_state_circuit = transpile(quantum_state_circuit, simulator_backend) 
    Qobj = assemble(transpiled_quantum_state_circuit)
    result = simulator_backend.run(Qobj).result()
    counts = result.get_counts(quantum_state_circuit)
    
    return get_probability_distribution(counts)

### Expectation value from probabilities

In [236]:
def sort_probas_dict_by_qubits_string_keys(proba_distribution: Dict) -> Dict:
    return dict(sorted(proba_distribution.items()))

def reset_power_of_minus_1(power_of_minus_1):
    power_of_minus_1 = 0
    return power_of_minus_1

def convert_pauli_string_into_str(pauli_string) -> str:
    return str(pauli_string)

def calculate_expectation_value_of_pauli_string_by_measurments_probas(pauli_string, ansatz_circuit):
    pauli_string_expectation_value = 0
    power_of_minus_1 = 0
    
    pauli_string_str = convert_pauli_string_into_str(pauli_string)
    extended_ansatz_circuit = add_layer_of_gates_for_reducing_paulis_to_sigma_z(pauli_string_str, ansatz_circuit)
    probas_distribution = calculate_probabilities_of_measurments_in_computational_basis(extended_ansatz_circuit)
    
    reduced_pauli_string = reduce_pauli_matrixes_into_sigma_z(pauli_string)
    sorted_probas_distribuition = sort_probas_dict_by_qubits_string_keys(probas_distribution)
    for qubits_string, proba in sorted_probas_distribuition.items():
        for string_index in range(QUBITS_NUM):
            if(str(qubits_string[string_index])=="1" and str(pauli_string[string_index])=="Z"):
                power_of_minus_1 += 1
            
        pauli_string_expectation_value += pow(-1, power_of_minus_1)*proba
        power_of_minus_1 = reset_power_of_minus_1(power_of_minus_1)
        
    return pauli_string_expectation_value

In [237]:
def get_expectation_value(ansatz_circuit, pauli_coeffs, pauli_strings):
    total_expection_value = 0
    
    for pauli_coeff, pauli_string in zip(pauli_coeffs, pauli_strings):
        total_expection_value += pauli_coeff*calculate_expectation_value_of_pauli_string_by_measurments_probas(
                                                                                    pauli_string, ansatz_circuit)
    
    return total_expection_value

## Objective Function

In [238]:
from qiskit import assemble, transpile

def cost_function(thetas, hamiltonian):
    L_w = 0
    circuit_params = prepare_circuit_params(thetas)
    computational_eigenvectors = get_first_k_eigenvectors_from_n_computational_basis(K, N)
    
    pauli_coeffs, pauli_strings = transfrom_hamiltonian_into_pauli_string(LiH_molecule_4_qubits)
    
    for j in tqdm(range(K)):
        ansatz_state = get_ansatz_state(circuit_params, computational_eigenvectors[j])
        approximated_energy = get_expectation_value(ansatz_state, pauli_coeffs, pauli_strings)
        insert_approximated_energy_to_list_of_all_approximated_energies(
            approximated_energies_dict["approximated_eneriges_"+str(j)], approximated_energy)
        L_w += w_vector[j]*approximated_energy
        
    return L_w

### Optimization

In [239]:
def get_optimal_thetas_of_ansatz_circuit_for_hamiltonian(hamiltonian):
    initial_thetas = np.random.uniform(low=0, high=2*np.pi, size=12)
    optimizer_result = minimize(cost_function,x0=initial_thetas,args=(hamiltonian),method="BFGS",options={"maxiter":NUM_ITERATIONS})
    optimal_thetas = prepare_circuit_params(optimizer_result.x)
    
    return optimal_thetas

In [240]:
def get_approximated_k_eigenvalues_of_hamiltonian(hamiltonian):
    approximated_k_eigenvalues = []
    optimal_thetas = get_optimal_thetas_of_ansatz_circuit_for_hamiltonian(hamiltonian)
    computational_eigenvectors = get_first_k_eigenvectors_from_n_computational_basis(K, N)
    
    pauli_coeffs, pauli_strings = transfrom_hamiltonian_into_pauli_strings(hamiltonian)
    for eigenvalue_index, eigenvector in enumerate(computational_eigenvectors):
        optimal_ansatz_state = get_ansatz_state(optimal_thetas, eigenvector)
        approximated_eigenvalue = get_expectation_value(optimal_ansatz_state, pauli_coeffs, pauli_strings)
        approximated_k_eigenvalues.append(approximated_eigenvalue)

    return approximated_k_eigenvalues

## Comparsion

In [241]:
from numpy import linalg as LA
from statistics import mean

def get_mean_approximation_error(exact_k_eigenvalues, approximated_k_eigenvalues):
    approximated_errors = []
    for exact_eigenvalue, approximated_eigenvalue in zip(exact_k_eigenvalues, approximated_k_eigenvalues):
        approximated_errors.append(abs(abs(exact_eigenvalue)-abs(approximated_eigenvalue))/abs(exact_eigenvalue))
        
    return mean(approximated_errors)

In [242]:
def get_exact_k_eigenvalues_of_hamiltonian(hamiltonian, k):
    eigenvalues = LA.eig(hamiltonian.to_matrix())[0]
    
    return sorted(eigenvalues,reverse=True)[:k]

In [243]:
def compare_exact_and_approximated_eigenvectors(hamiltonian, approximated_k_eigenvalues):
    exact_k_eigenvalues = get_exact_k_eigenvalues_of_hamiltonian(hamiltonian, K)
    print("Exact K Eigenvalues:")
    print(exact_k_eigenvalues)
    
    print("\nApproximated K Eigenvalues:")
    print(approximated_k_eigenvalues)

    print("\nMean Approximation error:")
    print(get_mean_approximation_error(exact_k_eigenvalues, approximated_k_eigenvalues))

## Visualization

In [244]:
def insert_approximated_energy_to_list_of_all_approximated_energies(approximated_energies_list, energy):
    approximated_energies_list.append(energy)

## LiH Molecule 4 qubits

In [245]:
from hamitonians import *

In [246]:
%%time
LiH_approximated_k_eigenvalues = get_approximated_k_eigenvalues_of_hamiltonian(LiH_molecule_4_qubits)

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:18<00:00,  4.74s/it]


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:11<00:00,  2.93s/it]


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:11<00:00,  2.89s/it]


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:11<00:00,  2.89s/it]


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:11<00:00,  2.98s/it]


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:12<00:00,  3.00s/it]


CPU times: total: 1h 28min 55s
Wall time: 1h 29min 51s


In [247]:
compare_exact_and_approximated_eigenvectors(LiH_molecule_4_qubits, LiH_approximated_k_eigenvalues)

Exact K Eigenvalues:
[(-6.769813218087976+0j), (-7.130406955301308+0j), (-7.130406955301309+0j), (-7.151525481896562+0j)]

Approximated K Eigenvalues:
[-7.620334218913603, -7.54610441440547, -7.486411159747651, -7.437668908979467]

Mean Approximation error:
0.06846818402803148


In [248]:
print(approximated_energies_dict)
approximated_energies_dict = { "approximated_eneriges_0": [],
"approximated_eneriges_1":[],
"approximated_eneriges_2": [],
"approximated_eneriges_3": [] }

{'approximated_eneriges_0': [-7.618997550637628, -7.610480926269329, -7.6173332886703315, -7.623505214696187, -7.610088338419853, -7.64478562914718, -7.622021571189597, -7.635477670881315, -7.626098520564665, -7.6271981294186775, -7.617633040410839, -7.615425457017947, -7.614435279234702, -7.596667679951785, -7.593371300726658, -7.609843631611862, -7.615520725001219, -7.600940447144158, -7.607593895124169, -7.608238596363764, -7.60407574929114, -7.611476224243539, -7.604786636578374, -7.61697364182984, -7.597380602767152, -7.618088655709216, -7.648136966842622, -7.637478153696063, -7.632750645774993, -7.620861256592474, -7.631521932163403, -7.623061948219196, -7.637408328558249, -7.626220360422885, -7.6178159319765655, -7.613673202016805, -7.6348507211365995, -7.635673278850026, -7.632269061959291, -7.624022078477773, -7.619296048022718, -7.617992397361088, -7.619112034492853, -7.611294753339254, -7.613126318591904, -7.627987001500941, -7.615553597688461, -7.617758051324575, -7.6407259

## H2 Molecule 4 qubits

In [249]:
%%time
H2_approximated_k_eigenvalues = get_approximated_k_eigenvalues_of_hamiltonian(H2_molecule_Hamiltonian_4_qubits)

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:11<00:00,  2.95s/it]


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:12<00:00,  3.00s/it]


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:21<00:00,  5.48s/it]


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:23<00:00,  5.95s/it]


CPU times: total: 54min 6s
Wall time: 54min 31s


In [250]:
compare_exact_and_approximated_eigenvectors(H2_molecule_Hamiltonian_4_qubits, H2_approximated_k_eigenvalues)

Exact K Eigenvalues:
[(0.2142782384194731+0j), (-1.942890293094024e-16+0j), (-0.2249112528308709+0j), (-0.353325104107155+0j)]

Approximated K Eigenvalues:
[-1.1129028032262829, -1.2843215824346217, -0.9070345133228698, -0.8835839516616046]

Mean Approximation error:
1652591485736130.0


In [251]:
print(approximated_energies_dict)
approximated_energies_dict = { "approximated_eneriges_0": [],
"approximated_eneriges_1":[],
"approximated_eneriges_2": [],
"approximated_eneriges_3": [] }

{'approximated_eneriges_0': [-7.684872016622813, -7.682674978622313, -7.688815045540018, -7.6865610070419885, -7.699764104952971, -7.683004522411683, -7.682182188775505, -7.6717369732692955, -7.692125194859612, -7.680169373542193, -7.6869262150081745, -7.683356540904024, -7.682711822500686, -7.672241543531918, -7.678109785918325, -7.674325745371277, -7.668429619630532, -7.663553759161611, -7.68778783075317, -7.68418286503949, -7.661996814177374, -7.668160670004843, -7.6616865622406936, -7.693219164537062, -7.686701530093904, -7.674359854212713, -7.744467843890669, -7.726744407323562, -7.74519766625444, -7.725407627703116, -7.742011824285679, -7.745011538786694, -7.748204435144161, -7.735741646569016, -7.731038912428485, -7.740322298346586, -7.751222703439206, -7.719490955831602, -7.742081876115709, -7.715101150037035, -7.724964388769998, -7.722782574876512, -7.704527442053856, -7.704680245249642, -7.705938869138346, -7.739088341237903, -7.711882080489956, -7.720023572479591, -7.7140355

##  Transverse Ising Model 4 qubits

In [252]:
%%time
TI_approximated_k_eigenvalues = get_approximated_k_eigenvalues_of_hamiltonian(transverse_ising_4_qubits)

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:11<00:00,  2.96s/it]


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:11<00:00,  2.99s/it]


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:11<00:00,  2.92s/it]


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:11<00:00,  2.95s/it]


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:12<00:00,  3.04s/it]


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:11<00:00,  2.86s/it]


CPU times: total: 1h 20min 33s
Wall time: 1h 21min


In [253]:
compare_exact_and_approximated_eigenvectors(transverse_ising_4_qubits, TI_approximated_k_eigenvalues)

Exact K Eigenvalues:
[(3.3650474360647804+0j), (3.071726889491433+0j), (1.7088632973761595+0j), (1.48288685094461+0j)]

Approximated K Eigenvalues:
[2.530115854944646, 2.547324377781065, 2.6747166610524924, 2.659707401667607]

Mean Approximation error:
0.4444103043165892


In [254]:
print(approximated_energies_dict)
approximated_energies_dict = { "approximated_eneriges_0": [],
"approximated_eneriges_1":[],
"approximated_eneriges_2": [],
"approximated_eneriges_3": [] }

{'approximated_eneriges_0': [-7.48610307706854, -7.486723782889051, -7.482810231578335, -7.493273576407161, -7.479828480352569, -7.502450598250617, -7.4895475489140875, -7.485024426068045, -7.494529240753221, -7.486915714782895, -7.4812406211037, -7.478072287140875, -7.486696394899587, -7.416186448755935, -7.428041212203513, -7.4275734586457425, -7.41179815275807, -7.433825802958651, -7.417792354534976, -7.4216050984979915, -7.434787792179803, -7.43567064928438, -7.4194222964554335, -7.4153862256994225, -7.417358134838512, -7.429091207884353, -7.405295351807469, -7.396285793953423, -7.39558984034964, -7.392024840992557, -7.397681167239909, -7.406928770991795, -7.405246126381449, -7.409210646950148, -7.410278824585284, -7.398014105741087, -7.398230928971183, -7.401017429293576, -7.387443597679132, -7.472074683872723, -7.464192146730993, -7.455751997199061, -7.468359802462868, -7.466412009624977, -7.478069399443806, -7.457613160986113, -7.462403342953777, -7.463661590168495, -7.443841990

In [113]:
initial_thetas = np.random.uniform(low=0, high=2*np.pi, size=12)
optimizer_result = minimize(cost_function,x0=initial_thetas,args=(LiH_molecule_4_qubits),method="BFGS",options={"maxiter":NUM_ITERATIONS})
print(optimizer_result)
optimal_thetas = prepare_circuit_params(optimizer_result.x)

computational_eigenvectors = get_first_k_eigenvectors_from_n_computational_basis(K, N)
optimal_ansatz_state = get_ansatz_state(optimal_thetas, computational_eigenvectors[K-1])
pauli_coeffs, pauli_strings = transfrom_hamiltonian_into_pauli_string(LiH_molecule_4_qubits)


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:09<00:00,  2.28s/it]


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:10<00:00,  2.60s/it]

      fun: -73.95773316051425
 hess_inv: array([[ 1.00307161e+00, -5.09201222e-02, -5.47356480e-02,
        -1.62681403e-01, -4.23696808e-02,  1.91395552e-01,
        -2.34563415e-01, -1.47399387e-02, -1.06994899e-01,
        -2.30856804e-01,  2.44203224e-01, -2.78136077e-03],
       [-5.09201222e-02,  8.96386378e-01, -9.54867683e-02,
        -1.71902098e-01, -6.07276602e-02,  9.78304586e-02,
        -2.22414166e-01, -6.01544432e-02, -1.33741982e-01,
        -2.23651086e-01,  1.34317906e-01, -3.42505201e-02],
       [-5.47356480e-02, -9.54867683e-02,  9.14173327e-01,
        -1.36682596e-01, -5.24829952e-02,  5.03213518e-02,
        -1.70152705e-01, -5.95562678e-02, -1.11781260e-01,
        -1.72491842e-01,  7.42529145e-02, -3.54695988e-02],
       [-1.62681403e-01, -1.71902098e-01, -1.36682596e-01,
         9.32466442e-01, -6.58881008e-02, -2.36598771e-01,
        -2.03558696e-02, -1.41056904e-01, -1.07021666e-01,
        -3.44168301e-02, -2.73137076e-01, -9.59325722e-02],
       [-4.

In [114]:
print(get_expectation_value(optimal_ansatz_state, pauli_coeffs, pauli_strings))
print(optimal_thetas)

-7.57072049639962
{'thetas': array([0.74760824, 0.17289584, 1.48511553, 2.87050967, 1.1083423 ,
       3.83152585, 4.61583078, 3.2915738 ]), 'phis': array([4.29275154, 5.47569434, 2.44077279, 1.02772525]), 'D1': 2, 'D2': 8}


In [115]:
pauli_coeffs, pauli_strings = transfrom_hamiltonian_into_pauli_string(LiH_molecule_4_qubits)
computational_eigenvectors = get_first_k_eigenvectors_from_n_computational_basis(K, N)
for eigenvalue_index, eigenvector in enumerate(computational_eigenvectors):
    print(eigenvector)
    optimal_ansatz_state = get_ansatz_state(optimal_thetas, eigenvector)
    print("eigenvalue for k =", eigenvalue_index)
    print(get_expectation_value(optimal_ansatz_state, pauli_coeffs, pauli_strings))

[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
eigenvalue for k = 0
-7.162165959477899
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
eigenvalue for k = 1
-7.517719632622515
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
eigenvalue for k = 2
-7.589285715860265
[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
eigenvalue for k = 3
-7.563737559574652


In [120]:
from numpy import linalg as LA
eigen_values = LA.eig(LiH_molecule_4_qubits.to_matrix())[0]
print(sorted(eigen_values, reverse=True))

[(-6.769813218087976+0j), (-7.130406955301308+0j), (-7.130406955301309+0j), (-7.151525481896562+0j), (-7.364817440287081+0j), (-7.511999706834451+0j), (-7.511999706834453+0j), (-7.569984737620559+0j), (-7.569984737620559+0j), (-7.700475837803976+0j), (-7.714056691660695+0j), (-7.714056691660695+0j), (-7.7140566916607005+0j), (-7.783396208286518+0j), (-7.7833962082865185+0j), (-7.862773163027979+0j)]
